<a href="https://colab.research.google.com/github/rootUJ99/hf_transformer_diffusers/blob/main/tuning_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip --version

!pip install --no-cache-dir --upgrade accelerate

pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)


In [54]:
!pip install datasets transformers torch accelerate -U

import transformers
import accelerate

transformers.__version__, accelerate.__version__


('4.31.0', '0.21.0')

In [41]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [48]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenizer_cb(sample):
    tokenizer(sample["text"], padding="max_length", truncation=True)


tokenized_dataset = dataset.map(tokenizer_cb, batched=True)

Map:   0%|          | 0/650000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [49]:
small_train_dataset = tokenized_dataset["train"].shuffle(seed=60).select(range(1000))
small_test_dataset = tokenized_dataset["test"].shuffle(seed=60).select(range(1000))




In [50]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

ImportError: ignored